# inforet 2

In [1]:
# 授权访问colab
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# no time to lose:
!wget https://gerdes.fr/saclay/informationRetrieval/our_msmarco.zip -O /content/drive/MyDrive/our_msmarco.zip
# this will be big: 1.2gb!
# you will get three files

--2024-03-17 13:29:43--  https://gerdes.fr/saclay/informationRetrieval/our_msmarco.zip
Resolving gerdes.fr (gerdes.fr)... 91.121.110.116
Connecting to gerdes.fr (gerdes.fr)|91.121.110.116|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 458004951 (437M) [application/zip]
Saving to: ‘/content/drive/MyDrive/our_msmarco.zip’

/content/drive/MyDr 100%[===================>] 436.79M  11.2MB/s    in 39s     

2024-03-17 13:30:22 (11.2 MB/s) - ‘/content/drive/MyDrive/our_msmarco.zip’ saved [458004951/458004951]



In [2]:
!unzip /content/drive/MyDrive/our_msmarco.zip -d /content/our_msmarco

Archive:  /content/drive/MyDrive/our_msmarco.zip
  inflating: /content/our_msmarco/our.msmarco.docs.tsv  
  inflating: /content/our_msmarco/our.msmarco.queries.tsv  
  inflating: /content/our_msmarco/our.msmarco.gold.tsv  


In [3]:
# this turns on the autotimer, so that every cell has a timing information below
try:
    %load_ext autotime
except:
    !pip install ipython-autotime
    %load_ext autotime
# stop using:
# %unload_ext autotime
# !pip install sklearn
# I delete "!pip install sklearn", because I encountered the problem:
  #  Downloading sklearn-0.0.post12.tar.gz (2.6 kB)
  #  error: subprocess-exited-with-error


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 15.0 MB/s eta 0:00:00
time: 453 µs (started: 2024-03-26 23:01:25 +00:00)


In [4]:
!pip install scikit-learn
import re
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from collections import Counter
from tqdm.notebook import tqdm
#from sklearn.feature_extraction.text import CountVectorizer
import dask.dataframe as dd

time: 7.65 s (started: 2024-03-26 23:01:25 +00:00)


## our dataset

- "TREC stands for the Text Retrieval Conference. Started in 1992 it is a series of workshops that focus on supporting research within the information retrieval community. It provides the infrastructure necessary for large-scale evaluation of text retrieval methodologies. Every year these workshops are organized, which are centered around a set of tracks. These tracks encourage new researches in the area of information retrieval."
- TREC 2019 Deep Learning Track https://microsoft.github.io/msmarco/TREC-Deep-Learning-2019
- data from MS-Marco https://microsoft.github.io/msmarco/
- The dataset contains  367k queries and a corpus of 3.2 million documents.
___
- if you want to reproduce my selection or get a bigger set, uncomment and execute


In [ ]:

# !wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.tsv.gz -O /content/drive/MyDrive/msmarco-docs.tsv.gz
# !wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain-queries.tsv.gz -O /content/drive/MyDrive/msmarco-doctrain-queries.tsv.gz
# !wget https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain-top100.gz -O /content/drive/MyDrive/msmarco-doctrain-top100.gz


--2024-03-15 13:06:43--  https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-docs.tsv.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 404 The specified resource does not exist.
2024-03-15 13:06:43 ERROR 404: The specified resource does not exist..

--2024-03-15 13:06:43--  https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain-queries.tsv.gz
Resolving msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)... 20.150.34.4
Connecting to msmarco.blob.core.windows.net (msmarco.blob.core.windows.net)|20.150.34.4|:443... connected.
HTTP request sent, awaiting response... 404 The specified resource does not exist.
2024-03-15 13:06:44 ERROR 404: The specified resource does not exist..

--2024-03-15 13:06:44--  https://msmarco.blob.core.windows.net/msmarcoranking/msmarco-doctrain

In [ ]:
# !gzip -d /content/drive/MyDrive/msmarco-docs.tsv.gz
# !gzip -d /content/drive/MyDrive/msmarco-doctrain-queries.tsv.gz
# !gzip -d /content/drive/MyDrive/msmarco-doctrain-top100.gz



gzip: /content/drive/MyDrive/msmarco-docs.tsv.gz: unexpected end of file

gzip: /content/drive/MyDrive/msmarco-doctrain-queries.tsv.gz: unexpected end of file

gzip: /content/drive/MyDrive/msmarco-doctrain-top100.gz: unexpected end of file
time: 323 ms (started: 2024-03-15 13:07:13 +00:00)


## skip to the section "reading in our smaller files"

come back here if you want to understand how to reduce the dataset or you want to build your own dataset

- we have three datasets:
    
    1. the queries: msmarco-doctrain-queries.tsv
    2. the gold: msmarco-doctrain-top100.tsv is a table containing query_id, doc_id and rank
    3. the actual documents: msmarco-docs.tsv 21GB of text! doc_id, url, title, text

In [ ]:
import os
os.chdir('/content/drive/MyDrive/')

time: 809 µs (started: 2024-03-22 12:03:07 +00:00)


In [ ]:
all_queries=pd.read_table('msmarco-doctrain-queries.tsv',header=None)
all_queries.columns=['qid','query']
print('Shape=>',all_queries.shape)
all_queries.head()

Shape=> (367013, 2)


,qid,query
0,1185869,)what was the immediate impact of the success ...
1,1185868,_________ justice is designed to repair the ha...
2,1183785,elegxo meaning
3,645590,what does physical medicine do
4,186154,feeding rice cereal how many times per day


time: 961 ms (started: 2024-03-15 13:50:43 +00:00)


#### reducing the dataset
- here we take 1000 queries.
- if this is too big for your computer, use this code to build smaller version, starting with the already reduced 1000 query set that we've downloaded before


In [ ]:
our_queries=all_queries.sample(n=1000,random_state=42).reset_index(drop=True)
print('Shape=>',our_queries.shape)
our_queries.head()

Shape=> (1000, 2)


,qid,query
0,687888,what is a jpe
1,480210,price for asphalt driveway
2,591004,what causes pressure skin bruising
3,260536,how long drive from flagstaff to grand canyon
4,39422,average number of bowel movements per day for ...


time: 62.8 ms (started: 2024-03-15 13:50:44 +00:00)


In [ ]:
our_queries.to_csv('our.msmarco.queries.tsv',sep='\t')

time: 18.6 ms (started: 2024-03-15 13:50:44 +00:00)


#### the gold file
- 36m lines!

In [ ]:
gold_top100=pd.read_table('msmarco-doctrain-top100',delimiter=' ',header=None)
gold_top100.columns=['qid','Q0','docid','rank','score','runstring']
print('Shape=>',gold_top100.shape)
display(gold_top100.head())
# Reducing train_top100 for training
# NameError: name 'train_top100' is not defined
# I changed it to gold_top100

our_gold_top100=gold_top100[gold_top100['qid'].isin(our_queries['qid'].unique())].reset_index(drop=True)
print('Shape=>',our_gold_top100.shape)
our_gold_top100.head()

Shape=> (36701116, 6)


,qid,Q0,docid,rank,score,runstring
0,1185869,Q0,D59221,1,-4.80433,IndriQueryLikelihood
1,1185869,Q0,D59220,2,-4.92127,IndriQueryLikelihood
2,1185869,Q0,D2192591,3,-5.05215,IndriQueryLikelihood
3,1185869,Q0,D2777518,4,-5.05486,IndriQueryLikelihood
4,1185869,Q0,D2371978,5,-5.07048,IndriQueryLikelihood


Shape=> (100000, 6)


,qid,Q0,docid,rank,score,runstring
0,310290,Q0,D579750,1,-5.11498,IndriQueryLikelihood
1,310290,Q0,D579754,2,-5.57703,IndriQueryLikelihood
2,310290,Q0,D2380815,3,-5.84852,IndriQueryLikelihood
3,310290,Q0,D822566,4,-5.95002,IndriQueryLikelihood
4,310290,Q0,D2249695,5,-6.08326,IndriQueryLikelihood


time: 1min 1s (started: 2024-03-15 13:50:51 +00:00)


In [ ]:
our_gold_top100.to_csv('our.msmarco.gold.tsv',sep='\t')

time: 910 ms (started: 2024-03-15 13:58:47 +00:00)


#### the data file

- it's so big that it's smarter to use dask: https://docs.dask.org/en/stable/

In [ ]:
!wget https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-docs.tsv.gz -O /content/msmarco-docs.tsv.gz

--2024-03-15 13:59:54--  https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-docs.tsv.gz
Resolving msmarco.z22.web.core.windows.net (msmarco.z22.web.core.windows.net)... 20.150.34.1
Connecting to msmarco.z22.web.core.windows.net (msmarco.z22.web.core.windows.net)|20.150.34.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8446274598 (7.9G) [application/x-gzip]
Saving to: ‘/content/msmarco-docs.tsv.gz’

/content/msmarco-do 100%[===================>]   7.87G  18.8MB/s    in 7m 17s  

2024-03-15 14:07:11 (18.4 MB/s) - ‘/content/msmarco-docs.tsv.gz’ saved [8446274598/8446274598]

time: 7min 17s (started: 2024-03-15 13:59:53 +00:00)


In [ ]:
!gzip -d /content/msmarco-docs.tsv.gz

time: 6min 13s (started: 2024-03-15 14:07:47 +00:00)


In [ ]:
df=dd.read_table('/content/msmarco-docs.tsv',blocksize=100e6,header=None) #  partitions of 100MB
df.columns=['docid','url','title','body']
df.head()

,docid,url,title,body
0,D1555982,https://answers.yahoo.com/question/index?qid=2...,The hot glowing surfaces of stars emit energy ...,Science & Mathematics Physics The hot glowing ...
1,D301595,http://childparenting.about.com/od/physicalemo...,Developmental Milestones and Your 8-Year-Old C...,School-Age Kids Growth & Development Developme...
2,D1359209,http://visihow.com/Check_for_Lice_Nits,Check for Lice Nits,Check for Lice Nits Edited by Mian Sheilette O...
3,D2147834,http://www.nytimes.com/2010/01/05/business/glo...,Dubai Opens a Tower to Beat All,Global Business Dubai Opens a Tower to Beat Al...
4,D1568809,http://www.realtor.com/realestateandhomes-sear...,"Coulterville, CA Real Estate & Homes for Sale","Coulterville, CA Real Estate & Homes for Sale4..."


time: 4.17 s (started: 2024-03-15 14:14:09 +00:00)


In [ ]:
# can't get the number of rows quickly :s
# very slow:
# len(df.index)

# faster:
os.chdir('/content/')
!wc -l msmarco-docs.tsv

3213835 msmarco-docs.tsv
time: 2min 51s (started: 2024-03-15 14:15:00 +00:00)


- big dataset with 3m rows!
- we want the top 100 for our queries
- this takes some time!

In [ ]:
def create_corpus(result):
  unique_docid=result['docid'].unique()
  condition=df['docid'].isin(unique_docid)
  corpus=df[condition].reset_index(drop=True)
  corpus=corpus.drop(columns='url')
  print('Number of Rows=>',len(corpus))
  return corpus

our_docs=create_corpus(our_gold_top100)
our_docs.head()

Number of Rows=> 92565


,docid,title,body
0,D2981241,What do you call a group of lions?,Lions Vocabulary of the English Language Word ...
1,D687756,.,"The A Priori Argument ( also, Rationalization;..."
2,D913099,Everything You Need To Learn How To Cook Veget...,Home > How To Cook Vegetables Everything You N...
3,D328017,"What is the difference between latitude, longi...",Longitude Latitude Geographic Coordinate Syste...
4,D1636347,When was the pulley invented?,Answers.com ® Wiki Answers ® Categories Techno...


time: 7min (started: 2024-03-15 14:18:08 +00:00)


In [ ]:
our_docs.to_csv('our.msmarco.docs.tsv',sep='\t', single_file=True)

- this is still a big file: 92k documents

# reading in our smaller files
here we use the

- !wget https://gerdes.fr/saclay/informationRetrieval/our_msmarco.zip
- !unzip our_msmarco.zip

In [5]:
import os
os.chdir('/content/our_msmarco')

time: 832 µs (started: 2024-03-26 23:01:45 +00:00)


In [6]:
queries = pd.read_csv('our.msmarco.queries.tsv',sep='\t',usecols=[1,2])
queries

,qid,query
0,687888,what is a jpe
1,480210,price for asphalt driveway
2,591004,what causes pressure skin bruising
3,260536,how long drive from flagstaff to grand canyon
4,39422,average number of bowel movements per day for ...
...,...,...
995,89597,cell voltage mv meaning
996,1167043,what an ip address
997,737304,what is daily max citizens atm
998,156934,do i need a florida commercial driver license


time: 48.4 ms (started: 2024-03-26 23:01:47 +00:00)


In [7]:
gold = pd.read_csv('our.msmarco.gold.tsv',sep='\t',usecols=[1,3,4,5])
gold

,qid,docid,rank,score
0,310290,D579750,1,-5.11498
1,310290,D579754,2,-5.57703
2,310290,D2380815,3,-5.84852
3,310290,D822566,4,-5.95002
4,310290,D2249695,5,-6.08326
...,...,...,...,...
99995,257942,D253854,96,-6.32693
99996,257942,D3056621,97,-6.32837
99997,257942,D1323491,98,-6.32871
99998,257942,D2722485,99,-6.33100


time: 168 ms (started: 2024-03-26 23:01:58 +00:00)


In [8]:
docs = pd.read_csv('our.msmarco.docs.tsv',sep='\t',usecols=[1,2,3])
docs

,docid,title,body
0,D2981241,What do you call a group of lions?,Lions Vocabulary of the English Language Word ...
1,D687756,.,"The A Priori Argument ( also, Rationalization;..."
2,D913099,Everything You Need To Learn How To Cook Veget...,Home > How To Cook Vegetables Everything You N...
3,D328017,"What is the difference between latitude, longi...",Longitude Latitude Geographic Coordinate Syste...
4,D1636347,When was the pulley invented?,Answers.com ® Wiki Answers ® Categories Techno...
...,...,...,...
92560,D3379210,Top 39 Doctor insights on: Can An Iud Cause Ha...,Top 39 Doctor insights on: Can An Iud Cause Ha...
92561,D3068739,How to get back your DirecTV cancellation fees,How to get back your Direc TV cancellation fee...
92562,D1590402,Certification FAQs,Fingerprinting 1. Where can I get fingerprinte...
92563,D2175490,Greenhouse gas emissions by Canadian economic ...,"Access PDF (682 KB)In 2015, Canada's total gre..."


time: 19 s (started: 2024-03-26 23:02:03 +00:00)


In [9]:
# Creating Training Set of Queries
training_queries=queries.iloc[:500]
print('Shape=>',training_queries.shape)
display(training_queries.head())
# Creating Testing Set of Queries
testing_queries=queries.iloc[500:]
print('Shape=>',testing_queries.shape)
testing_queries.head()

Shape=> (500, 2)


,qid,query
0,687888,what is a jpe
1,480210,price for asphalt driveway
2,591004,what causes pressure skin bruising
3,260536,how long drive from flagstaff to grand canyon
4,39422,average number of bowel movements per day for ...


Shape=> (500, 2)


,qid,query
500,116364,decree verb definition
501,638813,what does hemiballistic mean
502,401631,is advil considered aspirin?
503,1050265,who sang the song midnight train to georgia
504,632336,what does aq mean in chemistry


time: 23.6 ms (started: 2024-03-26 23:02:44 +00:00)


## exploring the data

### 🚧 todo: check whether there are NaN and take care of them

In [10]:
#... isna

print(f'There are {np.sum(queries["query"].isna())} NaN query in df.queries')


print(f'There are {np.sum(gold["score"].isna())} NaN score in df.golds')
print(f'There are {np.sum(docs["title"].isna())} NaN title and {np.sum(docs["body"].isna())} NaN body in df.docs')


There are 0 NaN query in df.queries
There are 0 NaN score in df.golds
There are 372 NaN title and 3 NaN body in df.docs
time: 27.3 ms (started: 2024-03-26 23:02:49 +00:00)


In [11]:
# ... fillna
docs["title"].fillna("NAN",inplace=True)
docs["body"].fillna("NAN",inplace=True)

time: 27.9 ms (started: 2024-03-26 23:02:51 +00:00)


### let's have a look at some random query:

In [12]:
queries.loc[111]

qid                                      251898
query    how long does getting a doctorate take
Name: 111, dtype: object

time: 5.31 ms (started: 2024-03-26 23:02:58 +00:00)


In [13]:
gold[gold.qid==251898]

,qid,docid,rank,score
36200,251898,D2865964,1,-4.74293
36201,251898,D3557816,2,-4.90695
36202,251898,D2723985,3,-4.95911
36203,251898,D1951655,4,-4.97272
36204,251898,D1709749,5,-5.02176
...,...,...,...,...
36295,251898,D2531901,96,-5.56896
36296,251898,D2956542,97,-5.57138
36297,251898,D301873,98,-5.57262
36298,251898,D2952336,99,-5.57504


time: 16.1 ms (started: 2024-03-26 23:03:01 +00:00)


### 🚧 todo: let's look at the top-ranked document for that query
- title
- body

In [ ]:
# todo: .values[0] can help

In [14]:
gold[gold.qid==251898].values[0]

array([251898, 'D2865964', 1, -4.74293], dtype=object)

time: 4.76 ms (started: 2024-03-26 23:03:08 +00:00)


In [15]:
gold[gold.qid==251898].values[0][1]

'D2865964'

time: 3.64 ms (started: 2024-03-26 23:03:10 +00:00)


In [16]:
# directly use the 'docid' I just got
#title
docs[docs.docid=='D2865964'].values[0][1]

'How long does it take to get a post-doctoral degree?'

time: 10.6 ms (started: 2024-03-26 23:03:11 +00:00)


In [17]:
# body
docs[docs.docid=='D2865964'].values[0][2]

"Answers.com ® Wiki Answers ® Categories Jobs & Education Education College Degrees Graduate Degrees How long does it take to get a post-doctoral degree? Flag How long does it take to get a post-doctoral degree? Answer by Joe Ragusa Confidence votes 98.6KI am not aware of any such thing as a postdoctoral degree. You may be referring to postdoctoral research which may be required for obtaining a tenure-track faculty position, especially at research oriented institutions. Some have suggested that postdoctoral appointments - that were traditionally optional - have become mandatory as demand for tenure-track positions in academia has drastically increased over previous decades (Wikipedia). The length of time in this case depends on the type, content, scope, and depth of the research.2 people found this useful Was this answer useful? Yes Somewhat No Tyler Durden9988 1,326 Contributions How long does it take to get a doctoral degree? It totally depends on the actual degree and school but you

time: 11.7 ms (started: 2024-03-26 23:03:13 +00:00)


In [18]:
# Or
docs[docs.docid == gold[gold.qid==251898].values[0][1]].values[0]

array(['D2865964', 'How long does it take to get a post-doctoral degree?',
       "Answers.com ® Wiki Answers ® Categories Jobs & Education Education College Degrees Graduate Degrees How long does it take to get a post-doctoral degree? Flag How long does it take to get a post-doctoral degree? Answer by Joe Ragusa Confidence votes 98.6KI am not aware of any such thing as a postdoctoral degree. You may be referring to postdoctoral research which may be required for obtaining a tenure-track faculty position, especially at research oriented institutions. Some have suggested that postdoctoral appointments - that were traditionally optional - have become mandatory as demand for tenure-track positions in academia has drastically increased over previous decades (Wikipedia). The length of time in this case depends on the type, content, scope, and depth of the research.2 people found this useful Was this answer useful? Yes Somewhat No Tyler Durden9988 1,326 Contributions How long does it take to

time: 11.7 ms (started: 2024-03-26 23:03:14 +00:00)


### 🚧 todo: let's look at the second document
- let's make a functioin to make that easier

In [19]:
def titleAndBody(qid,rank):
    display(docs[docs.docid == gold[gold.qid==qid].values[rank][1]].values[0][1])
    display(docs[docs.docid == gold[gold.qid==qid].values[rank][1]].values[0][2])
titleAndBody(251898,1)

'How long does it take to get a degree in psychology?'

"Answers.com ® Wiki Answers ® Categories Jobs & Education Education College Degrees Bachelors Degrees How long does it take to get a degree in psychology? Flag How long does it take to get a degree in psychology? Answer by Colette Fisher Stone Confidence votes 16To complete a bachelors degree, it can take anywhere from 124 -128 credits. This could take approximately four years of study, provided the student takes the program as prescribed by the institution. Viper1 I just finished my Bachelor's in Psychology. It actually takes anywhere from 180-195 credits, and does take four years if you are taking 2 classes a semester. If you take on a heavier class load, it is possible to graduate in 3 years.15 people found this useful Was this answer useful? Yes Somewhat No Joe Ragusa How long does it take to earn a bachelor's degree in forensic psychology? It usually takes about 4 years to receive a Bachelor's Degree Answer I might also add that it depends on entrance testing of basic skills, cred

time: 20.5 ms (started: 2024-03-26 23:03:18 +00:00)


#### let's look at the 100th document

In [20]:
titleAndBody(251898,99)

'.'

'Q How long to receive va dic benefits? I receive a widows benefit from railroad retirement can i collect ssame time Topic: Asked by: Lino In Education & Reference > Financial Aid > Receive>A Top Solutions Janice sibug of 1631 el camino real #7 tustin ca 92780 was born to an illegal alien name l ... read more If direct deposit keep checking your bank account. I received my award letter (explaning b ... read more Add your answer Post to Facebook Post to Twitter Subscribe me Suggested Solutions (10) What\'s this? Anonymous0 2 Janice sibug of 1631 el camino real #7 tustin ca 92780 was born to an illegal alien name ligaya fabian who jumpshipped her flght at lax from germany to canada.source: How much do anchor babies receive in benefits? Was this answer helpful? Yes | No Comment Reply Reportpgfua Level 2 (Sophomore)4 Answers"If direct deposit keep checking your bank account..."5 0 If direct deposit keep checking your bank account. I received my award letter (explaning benefits) after I rec

time: 19.5 ms (started: 2024-03-26 23:03:24 +00:00)


### 🚧 todo: try this with a few different queries to get a feel of the quality of the gold

# doing our first baseline retrieval function

- todo:
    - build and fit a binary CountVectorizer on the **titles**
    - play with and understand build_analyzer, build_tokenizer, and transform
    - transform our query 111
        - understand what happens with yet unseen words in the transform process
    - find the docs with the most words in common
    - write an evaluation function computing the top 10 precision p@10
    - apply to our 500 queries


In [21]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(binary=True)
# understand the options:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
X = vectorizer.fit_transform(docs.title.tolist())
print('we got',len(vectorizer.get_feature_names_out()),'features, for example',vectorizer.get_feature_names_out()[33333:33339])

we got 39560 features, for example ['spurweed' 'sputtering' 'sputters' 'sputum' 'spy' 'spyware']
time: 785 ms (started: 2024-03-26 23:03:32 +00:00)


Since I met the problem: 'CountVectorizer' object has no attribute 'get_feature_names', I changed it to 'get_feature_names_out' to match the version of sklearn I installed before.

In [22]:
queries.loc[111].query


'how long does getting a doctorate take'

time: 2.79 ms (started: 2024-03-26 23:03:37 +00:00)


In [23]:
vectorizer.build_analyzer()(queries.loc[111].query)
# Return a callable to process input data.
# The callable handles preprocessing, tokenization, and n-grams generation.

['how', 'long', 'does', 'getting', 'doctorate', 'take']

time: 2.87 ms (started: 2024-03-26 23:03:39 +00:00)


In [24]:
vectorizer.build_tokenizer()(queries.loc[111].query)
# Return a function that splits a string into a sequence of tokens.

['how', 'long', 'does', 'getting', 'doctorate', 'take']

time: 2.8 ms (started: 2024-03-26 23:03:42 +00:00)


In [25]:
qv = vectorizer.transform([queries.loc[111].query])
qv

<1x39560 sparse matrix of type '<class 'numpy.int64'>'
	with 6 stored elements in Compressed Sparse Row format>

time: 3.36 ms (started: 2024-03-26 23:03:45 +00:00)


### 🚧 todo:
- understand what happens with yet unseen words in the transform process


- think of the shape of X, what are the rows, what are the columns?
- how to select the titles that have the words of our query?
- think of matrix multiplication and transposition

In [29]:
xqv = qv.toarray()
xqv

array([[0, 0, 0, ..., 0, 0, 0]])

time: 2.9 ms (started: 2024-03-26 23:04:09 +00:00)


In [30]:
xqv.shape

(1, 39560)

time: 2.67 ms (started: 2024-03-26 23:04:12 +00:00)


In [31]:
X.shape

(92565, 39560)

time: 2.13 ms (started: 2024-03-26 23:04:13 +00:00)


In [32]:
# Perform matrix multiplication between the matrix X and the transposed query matrix xqv.
outcome = X @ xqv.transpose()
print(outcome)

[[0]
 [0]
 [1]
 ...
 [0]
 [0]
 [0]]
time: 3.25 ms (started: 2024-03-26 23:04:16 +00:00)


In [33]:
outcome.shape

(92565, 1)

time: 2.5 ms (started: 2024-03-26 23:04:18 +00:00)


In [34]:
# Select the titles that have at least one of the words in the query
select_titles = [title for i, title in enumerate(docs.title.tolist()) if outcome[i]>=1]

time: 131 ms (started: 2024-03-26 23:04:20 +00:00)


In [35]:
print(select_titles[:5])

['Everything You Need To Learn How To Cook Vegetables', 'How tall are penguins?', 'Holly C.Size: 105Graded potentialsmay be either a depolarization or a hyperpolarization.Which of these is the earliest step in the generation of an action potential?Sodium channels open.The most abundant class of neuron in the central nervous system isMultipolar___ channels open or close in response to physical distortion of the membrane surface.Mechanically gatedClusters of RER and free ribosomes in neurons are calledNissl BodiesAction potential propagation begins (is first generated at) what region of a neuron?initial segmentWhere are action potentials regenerated as they propagate along an unmyelinated axon?The movement of what ion is responsible for the local currents that depolarize other regions of the axon to threshold? In an unmyelinated axon, why doesn\'t the action potential suddenly "double back" and start propagating in the opposite direction?Approximately how fast do action potentials propag

### 🚧 todo:
  - look at argmax and max,
  - check the numpy
      - flatnonzero function to find the best match
      - the .A and the .flat functions
  - show the best matching doc

1. argmax 和 max

- max函数：用于计算数组或矩阵中的最大值。

- argmax函数：用于找出数组或矩阵中最大值的索引位置。在寻找最佳匹配文档的场景中，如果你有一个由相似度分数组成的数组，argmax可以帮助你确定哪个文档（通过索引）具有最高的相似度分数。

2. numpy.flatnonzero 函数

- numpy.flatnonzero：这个函数返回输入数组中非零元素的索引。在寻找最佳匹配的上下文中，如果你有一个布尔数组或任何可以评估为True（非零）或False（零）的条件，flatnonzero可以帮助你找到满足条件的元素的索引。这对于过滤或选择特定条件的文档可能很有用。

3. .A 和 .flat 方法

- .A：这是在处理稀疏矩阵时常见的方法，用于获取稀疏矩阵的密集表示（即将其转换为一个常规的NumPy数组）。这在你需要对矩阵执行操作但库只接受密集格式时很有用。

- .flat：这是一个迭代器，用于以一维迭代的方式访问数组或矩阵的元素。它可以用来遍历矩阵的所有元素，就像它们是在一个单一的数组中一样。

In [36]:
# the index of the document with the most common words
max_idx = np.argmax(outcome)
print(f'The index of the document with the most common words is {max_idx}')

# the length of the most common words of the best matching document
max_num = outcome[max_idx][0]
print(f'The length of the most common words is {max_num}')

The index of the document with the most common words is 11957
The length of the most common words is 6
time: 1.45 ms (started: 2024-03-26 23:04:27 +00:00)


In [37]:
# Check the numpy
# numpy.flatnonzero
print(np.flatnonzero(outcome))

[    2     7     8 ... 92552 92558 92561]
time: 2.04 ms (started: 2024-03-26 23:04:31 +00:00)


In [38]:

# .flat
outcome.flat[11957]

6

time: 3.08 ms (started: 2024-03-26 23:04:33 +00:00)


In [39]:
docs.loc[max_idx].body

'Education & Reference Higher Education (University +)After getting my bachelors how long does it take to get a doctorate? In my last year of my bachelors degree in psychology and I want to get a doctorate. But I have no idea how long it would take. And is it better to get a masters a doctorate? Follow 6 answers Answers Relevance Rating Newest Oldest Best Answer: It generally takes 2 years to complete a master\'s and after that I would say at least 3-4 to complete a Ph. D. although many doctorates take much longer than that: sometimes up to 6-8 years. Unless you\'re trying to get into a unique medical field, such as virology or epidemiology etc., or to work/teach at the college level, then a master\'s degree will suffice for the most part. In fact, some colleges employ professors who hold a Masters degree and don\'t have a doctorate. also fyi, for every 100 Ph. D.\'s rewarded to a person, there are fewer than 10 collegiate job openings :). Hope that helped :)dvd_clapp · 10 years ago0 1

time: 2.48 ms (started: 2024-03-26 23:04:35 +00:00)


### 🚧 todo: use argpartition to get the 10 best answers

[numpy.argpartition](https://https://numpy.org/doc/stable/reference/generated/numpy.argpartition.html)(a, kth, axis=-1, kind='introselect', order=None)


Perform an indirect partition along the given axis using the algorithm specified by the kind keyword. It returns an array of indices of the same shape as a that index data along the given axis in partitioned order.



In [40]:
pred10i = np.argpartition(outcome.ravel(),-10)[-10:]
pred10i
# np.ravel()多维数组展平成一维数组

array([16630, 91275, 44359, 71292,  1477, 43190, 66313,  3686, 83787,
       11957])

time: 3.76 ms (started: 2024-03-26 23:04:39 +00:00)


In [41]:
docs.loc[pred10i]

,docid,title,body
16630,D3070882,How long does it take a tattoo to heal?,Answers.com ® Wiki Answers ® Categories Health...
91275,D3090762,How Long Does It Take to Get a Doctorate in En...,Home » College How Long Does It Take to Get a ...
44359,D638869,How long does it take for the femur bone to heal,Home > Surgery >How long does it take for the ...
71292,D471734,How long does it take to earn an marketing deg...,Expert answer by Joe Ragusa Confidence votes 9...
1477,D2813027,How long does Seroquel stay in your system if ...,Answers.com ® Wiki Answers ® Categories Health...
43190,D700799,Holidays By Country Â»Holidays By Religion Â»O...,Home > Important Days > Daylight Savings > Day...
66313,D2956542,How long does it take to finish a doctorate?,Education & Reference Higher Education (Univer...
3686,D2111066,After getting a flu shot how long does it take...,Celticballa568 19 Contributions After getting ...
83787,D1951655,How long does it take to get a Doctorate of Nu...,Answers.com ® Wiki Answers ® Categories Jobs &...
11957,D2408424,After getting my bachelors how long does it ta...,Education & Reference Higher Education (Univer...


time: 22.4 ms (started: 2024-03-26 23:04:43 +00:00)


In [42]:
docs.loc[pred10i].docid

16630    D3070882
91275    D3090762
44359     D638869
71292     D471734
1477     D2813027
43190     D700799
66313    D2956542
3686     D2111066
83787    D1951655
11957    D2408424
Name: docid, dtype: object

time: 4.53 ms (started: 2024-03-26 23:04:46 +00:00)


### 🚧 todo:
- find the relevant documents that are in our top 10
- user intersect1d
- compute the precision p@10

[numpy.intersect1d](https://numpy.org/doc/stable/reference/generated/numpy.intersect1d.html)(ar1, ar2, assume_unique=False, return_indices=False)

Find the intersection of two arrays.

Return the sorted, unique values that are in both of the input arrays.

In [43]:
relevant_doc = gold[gold.qid==251898].docid.tolist()

top10 = docs.loc[pred10i].docid.tolist()

time: 2.5 ms (started: 2024-03-26 23:04:50 +00:00)


In [44]:
intersection = np.intersect1d(top10, relevant_doc)
intersection

array(['D1951655', 'D2408424', 'D2956542', 'D3090762', 'D471734'],
      dtype='<U8')

time: 3.41 ms (started: 2024-03-26 23:04:51 +00:00)


In [45]:
precision = len(intersection)/10
precision

0.5

time: 3.36 ms (started: 2024-03-26 23:04:55 +00:00)


In [46]:
# 🚧 todo: build a function p@10 that gives the precision at 10
def pAt10(qid):
    qv = vectorizer.transform(queries[queries.qid==qid]['query'].tolist()).toarray()
    outcome = X @ qv.transpose()
    pred10i = np.argpartition(outcome.ravel(),-10)[-10:]

    relevant_doc = gold[gold.qid==qid].docid.tolist()
    top10 = docs.loc[pred10i].docid.tolist()

    intersection = np.intersect1d(top10, relevant_doc)
    precision = len(intersection)/10
    return precision

pAt10(251898)


0.5

time: 13.7 ms (started: 2024-03-26 23:04:57 +00:00)


### 🚧 todo:
- take our 500 training queries qid
- apply our function
- compute the average

In [47]:
training_queries.qid

0      687888
1      480210
2      591004
3      260536
4       39422
        ...  
495    133970
496     79788
497    791583
498    732078
499    197098
Name: qid, Length: 500, dtype: int64

time: 4.31 ms (started: 2024-03-26 23:05:01 +00:00)


In [48]:
training_queries.qid.apply(pAt10)

0      0.0
1      0.5
2      0.3
3      0.5
4      0.4
      ... 
495    0.6
496    0.1
497    0.0
498    0.1
499    0.3
Name: qid, Length: 500, dtype: float64

time: 1.65 s (started: 2024-03-26 23:05:03 +00:00)


In [49]:
training_queries.qid.apply(pAt10).mean()

0.32

time: 1.66 s (started: 2024-03-26 23:05:07 +00:00)


- that looks like a baseline we can beat :)
- what's the query we are doing best in?
    - max?

In [50]:
training_queries.qid.apply(pAt10).max()

1.0

time: 1.63 s (started: 2024-03-26 23:05:10 +00:00)


- oh, we have just been lucky before...

## 🚧 todo:

- redo the vectorization and evaluation on the whole text, not only the titles
- try the non-binary CountVectorizer
- go for tf-idf
    - play with at least two options and re-evaluate
- find other improvements. these may include:
    - cleaning the text
    - heuristically combining title and body matches
    - looking at bigrams
    - looking at terms (by means of a clean multi-word term list from wikipedia, see notebook 1)
    - by removing stopwords (look at nltk or spacy to do that)
    - trying an implementation of bm25 (slow with standard implementation, see last cell if you want to do it)
  
- do a grid search with a few promising parameters
    - maybe get inspired by GridSearchCV and pipelines in https://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html#sphx-glr-auto-examples-model-selection-grid-search-text-feature-extraction-py
        - you can also check the weel-written section "Pipelines" in this book: https://www.oreilly.com/library/view/applied-text-analysis/9781491963036/ch04.html
    - make a nice visualization of the results
    
- interpret the complete results in 3 to 5 sentences.
    - what strategy would do best if we switch our evaluation to p@100?

- give some ideas for improving the results





#### Redo the vectorization and evaluation on the whole text, not only the titles

In [ ]:
# Tokenization
vectorizer = CountVectorizer(binary=True)
X = vectorizer.fit_transform(docs.body.tolist())

# evaluation
training_queries.qid.apply(pAt10).mean()

0.48

time: 4min 33s (started: 2024-03-22 13:25:07 +00:00)


#### Try the non-binary CountVectorizer

In [ ]:
# Tokenization
vectorizer = CountVectorizer(binary=False)
X = vectorizer.fit_transform(docs.body.tolist())

# evaluation
training_queries.qid.apply(pAt10).mean()

0.046200000000000005

time: 4min 52s (started: 2024-03-17 14:39:27 +00:00)


#### Go for tf-idf

Here I use 2 different normalization methods

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# vectorization
vectorizer = TfidfVectorizer(norm='l1')
X = vectorizer.fit_transform(docs.body.tolist())

# evaluation
training_queries.qid.apply(pAt10).mean()

0.7766000000000001

time: 4min 22s (started: 2024-03-17 14:44:19 +00:00)


In [ ]:
# vectorization
vectorizer = TfidfVectorizer(norm='l2')

X = vectorizer.fit_transform(docs.body.tolist())

# evaluation
training_queries.qid.apply(pAt10).mean()

0.8972

time: 4min 18s (started: 2024-03-17 14:48:42 +00:00)


#### Find other improvements

1. cleaning the text

In [51]:
# Function for leaning text
# remove words with numbers inside
# replace new lines by space
# remove urls
# only keep ascii words
def clean_text(text):
    # remove words with numbers inside
    text=re.sub('\w*\d+\w*','', text)
    # replace new lines by space
    text=text.replace('\n',' ')
    # remove urls
    text = re.sub(r'http\S+', '', text)
    # only keep ascii words
    text = text.encode("ascii", errors="ignore").decode()
    return text

# Cleaning corpus using RegEx
docs['cleaned']=docs['body'].apply(lambda x: clean_text(x))


time: 3min 6s (started: 2024-03-26 23:05:26 +00:00)


In [ ]:
docs

,docid,title,body,cleaned
0,D2981241,What do you call a group of lions?,Lions Vocabulary of the English Language Word ...,Lions Vocabulary of the English Language Word ...
1,D687756,.,"The A Priori Argument ( also, Rationalization;...","The A Priori Argument ( also, Rationalization;..."
2,D913099,Everything You Need To Learn How To Cook Veget...,Home > How To Cook Vegetables Everything You N...,Home > How To Cook Vegetables Everything You N...
3,D328017,"What is the difference between latitude, longi...",Longitude Latitude Geographic Coordinate Syste...,Longitude Latitude Geographic Coordinate Syste...
4,D1636347,When was the pulley invented?,Answers.com ® Wiki Answers ® Categories Techno...,Answers.com Wiki Answers Categories Technolo...
...,...,...,...,...
92560,D3379210,Top 39 Doctor insights on: Can An Iud Cause Ha...,Top 39 Doctor insights on: Can An Iud Cause Ha...,Top Doctor insights on: Can An Iud Cause Hair...
92561,D3068739,How to get back your DirecTV cancellation fees,How to get back your Direc TV cancellation fee...,How to get back your Direc TV cancellation fee...
92562,D1590402,Certification FAQs,Fingerprinting 1. Where can I get fingerprinte...,Fingerprinting . Where can I get fingerprinted...
92563,D2175490,Greenhouse gas emissions by Canadian economic ...,"Access PDF (682 KB)In 2015, Canada's total gre...","Access PDF ( KB)In , Canada's total greenhouse..."


time: 15.5 s (started: 2024-03-22 13:37:33 +00:00)


2. heuristically combining title and body matches
looking at bigrams


In [55]:
# combine title and body
docs['title_body'] = docs['title'] + ' ' + docs['body']


time: 1.53 s (started: 2024-03-26 23:16:21 +00:00)


In [ ]:
vectorizer = CountVectorizer(ngram_range=(2,2))
X_tb = vectorizer.fit_transform(docs['title_body'].tolist())

time: 7min 26s (started: 2024-03-22 14:28:45 +00:00)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_similarity()

3. looking at terms (by means of a clean multi-word term list from wikipedia, see notebook 1)



In [ ]:
# TfidfVectorizer(sublinear_tf=True, strip_accents='unicode') 最好的结果
# fit_transform(docs['title']+' '+ docs['body'])

4. by removing stopwords (look at nltk or spacy to do that)
5. trying an implementation of bm25 (slow with standard implementation, see last cell if you want to do it)

In [ ]:
class BM25Score(BaseEstimator, ClassifierMixin):
    def init(self, vectorized_docs, k1=1.5, b=0.75): # vectorized_docs: word counts
        self.k1 = k1
        self.b = b
        self.vectorized_docs = vectorized_docs
    def fit(self, vectorized_queries=None, query_ids=None, args): # args are for grid search integration
        self.n_d = self.vectorized_docs.sum(axis=1).reshape(-1, 1).A
        self.avgdl = np.mean(self.n_d)
        self.n_docs = self.vectorized_docs.shape[0]

        self.nq = np.sum(self.vectorized_docs > 0, axis=0).reshape(1, -1)
        self.idf = np.log(((self.n_docs - self.nq + 0.5) / (self.nq + 0.5)) + 1).A
        return self
    def predict(self, vectorized_queries):
        # bigger batches => faster but more memory heavy
        n_queries = vectorized_queries.shape[0]
        batch_size = 50

        final_scores = []
        for batch_idx in range(int(n_queries / batch_size) + 1):
            start_idx = batch_idx batch_size
            end_idx = min((batch_idx + 1) * batch_size, n_queries)
            if start_idx >= end_idx:
                break
            batch_vectorized_queries = vectorized_queries[start_idx: end_idx]
            idx_tokens = np.argwhere(batch_vectorized_queries.sum(axis=0) > 0).reshape(-1)
            batch_vectorized_queries = batch_vectorized_queries[:, idx_tokens]
            vectorized_docs = self.vectorized_docs[:, idx_tokens].toarray()
            idf = self.idf[:, idx_tokens]

            scores = idf * (
                (vectorized_docs * (self.k1 + 1)) / \
                (
                    vectorized_docs + \
                    self.k1 * (1 - self.b + self.b * (self.n_d / self.avgdl))
                )
            )
            final_scores.append((batch_vectorized_queries > 0).astype(np.int8) @ scores.T)

            del vectorized_docs
            del scores
            gc.collect()
        return np.concatenate(final_scores, axis=0)